## Classification task


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import sys
import matplotlib.pyplot as plt
from tsne_torch import TorchTSNE
import numpy as np
sys.path.append('/home/gridsan/ckoevesdi/.local/lib/python3.9/site-packages/')
from pytorch_metric_learning import losses

sys.path.append('/home/gridsan/ckoevesdi/PooledStatisticsMetamers/poolstatmetamer/')
import utils.statnetencoder as sne
import importlib
import imp
import torch.nn.functional as F

#!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py

#sys.path.append(r'C:\Users\chris\Documents\MIT\Statistics_analysis_code\PyTorchSteerablePyramid')
sys.path.append('/home/gridsan/ckoevesdi/PyTorchSteerablePyramid/')
import steerable
import steerable.utils as utils
from steerable.SCFpyr_PyTorch import SCFpyr_PyTorch

torch.manual_seed(16)

#use GPU 2
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


In [2]:
dtd_folder = '/home/gridsan/ckoevesdi/data/dtd_torch/dtd/dtd/'

## Creation of the dataset

In [3]:
loading_transforms = transforms.Compose([
    transforms.CenterCrop(size=300),
    transforms.Grayscale(),
    transforms.ToTensor(),  # This will also convert the image from [0, 255] to [0.0, 1.0]
    transforms.ConvertImageDtype(torch.float32)
])

#use training set for now
class_dtd_dataset = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/dtd_torch/', split='val', 
                                             partition=10, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen
# Define the batch size (Change this based on your requirements)
batch_size = 1

# Create a DataLoader
classification_dataloader = DataLoader(
    class_dtd_dataset,
    batch_size=batch_size,
    shuffle=True
)
tensor2pil_transform = transforms.ToPILImage()
print(len(class_dtd_dataset))

939


In [4]:
if(False):
    for j, (texture_batch, labels) in enumerate(classification_dataloader):
            # Move data to device and flatten it
            print(texture_batch)
            print(labels)
            output = texture_batch.to(device)
            print(output.shape)
            for i in range(batch_size):
                    plt.figure()
                    plt.imshow(tensor2pil_transform(output[0,0,:,:]),cmap='gray')
                    plt.axis('off')
                    plt.show()
            if(j>3):
                break;

In [5]:
import utils.brucenet as bn
brucy = bn.BruceNet(pooling_region_size=1e20, pyramid_params=False, dummy_img = torch.zeros(20,
                                     1,
                                     300,
                                     300)).to(device)

Creating solver: pyramid=UBBBBL_6 pooling=WholeImagePooling()


/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
all_statistics = []

for j, (texture_batch, labels) in enumerate(classification_dataloader):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    all_statistics.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })


In [7]:
print(all_statistics[50])

{'statistics': array([ 6.30244672e-01,  1.54877827e-01,  1.60244368e-02,  2.18463764e-01,
        1.15038104e-01,  6.33021072e-02,  3.04496050e-01,  1.57857299e-01,
        8.57136175e-02,  4.17947620e-01,  2.09199965e-01,  1.09768219e-01,
        5.69129884e-01,  2.71880656e-01,  1.35975912e-01,  7.70545185e-01,
        3.49232107e-01,  1.64992988e-01,  2.74513751e-01,  9.54570025e-02,
        1.55887395e-01,  2.83173155e-02,  4.13577020e-01,  1.85625806e-01,
        2.75265217e-01,  3.91351394e-02,  3.89543116e-01,  1.73685029e-01,
        2.80285537e-01,  4.92304303e-02,  2.20321164e-01,  4.96202409e-02,
        9.71030742e-02,  1.49331884e-02,  1.03997104e-01,  7.66281188e-02,
        5.25283813e-02,  1.51419520e-01,  7.26732090e-02,  7.62581602e-02,
        6.14765644e-01,  2.67693371e-01,  3.80115896e-01,  1.54476583e-01,
        8.28454852e-01,  4.64545786e-01,  4.82536107e-01,  3.10085833e-01,
        8.57578456e-01,  4.99076843e-01,  6.17457628e-01,  2.92146415e-01,
        5.

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# Convert the list of dictionaries to a NumPy array or PyTorch tensor
statistics_array = np.array([item['statistics'] for item in all_statistics])
labels_array = np.array([item['original_image_index'] for item in all_statistics])
print(statistics_array.shape)
# Standardize
scaler = StandardScaler()
scaled_array = scaler.fit_transform(statistics_array)

# Convert NumPy arrays to PyTorch tensors
statistics_tensor = torch.tensor(scaled_array, dtype=torch.float32)
labels_tensor = torch.tensor(labels_array, dtype=torch.long)
print(statistics_tensor[2, 50])

(939, 150)
tensor([ 0.6732,  0.5422,  0.3349,  0.5369,  0.3981,  0.2791,  0.5205,  0.3594,
         0.2196,  0.4876,  0.2863,  0.1111,  0.4892,  0.2805,  0.0941,  0.5167,
         0.3269,  0.1496,  1.4103,  0.8753,  0.9940,  0.7747,  1.4585,  0.9613,
         1.1522,  0.8031,  0.8904,  0.3291,  0.5801,  0.3489,  1.0333,  0.4893,
         0.7976,  0.4762,  1.0264,  0.7965,  0.7382,  0.6431,  0.7622,  0.4163,
         1.5174,  0.9432,  0.5800,  0.8578,  1.5895,  1.0989,  0.5598,  0.8178,
         1.3301,  0.7296,  0.3289,  0.6552,  1.5067,  0.9389,  0.4592,  0.7414,
         1.1528,  1.3396,  1.0931,  1.0230,  1.1643,  0.9132,  0.6266,  0.1804,
         0.0521, -0.0978,  0.4873,  0.0607, -0.1757, -0.1980,  0.3323, -0.0611,
        -0.1530, -0.2909,  0.4535,  0.0305, -0.1991, -0.1392,  0.1664,  0.2906,
         0.1492,  0.0356,  0.1046,  0.0126, -0.0792, -0.1445, -0.3838, -0.2528,
        -0.3743, -0.5031, -0.2109, -0.2669, -0.4068, -0.3035, -0.3936, -0.5036,
        -0.2767, -0.3663, -0.

In [11]:
print(statistics_tensor[2, :49])

tensor([0.6732, 0.5422, 0.3349, 0.5369, 0.3981, 0.2791, 0.5205, 0.3594, 0.2196,
        0.4876, 0.2863, 0.1111, 0.4892, 0.2805, 0.0941, 0.5167, 0.3269, 0.1496,
        1.4103, 0.8753, 0.9940, 0.7747, 1.4585, 0.9613, 1.1522, 0.8031, 0.8904,
        0.3291, 0.5801, 0.3489, 1.0333, 0.4893, 0.7976, 0.4762, 1.0264, 0.7965,
        0.7382, 0.6431, 0.7622, 0.4163, 1.5174, 0.9432, 0.5800, 0.8578, 1.5895,
        1.0989, 0.5598, 0.8178, 1.3301])


In [9]:
from torch.utils.data import Dataset, DataLoader

class StatisticsDataset(Dataset):
    def __init__(self, statistics_tensor, labels_tensor):
        self.statistics = statistics_tensor
        self.labels = labels_tensor
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.statistics[idx], self.labels[idx]

# Create the dataset object
statistics_dataset = StatisticsDataset(statistics_tensor, labels_tensor)

batches = 80
class_statistics_dataloader = DataLoader(statistics_dataset, batch_size=batches, shuffle=True)

Checking does it make sense to even try? Intra-class variation statistics wise

In [21]:
m = statistics_tensor[14]
m2 = statistics_tensor[5]
l = statistics_tensor[90]

distance12 = torch.norm(m-m2).item()
distance13 = torch.norm(m-l).item()
distance23 = torch.norm(m2-l).item()
print(distance12)
print(distance13)
print(distance23)

25.405881881713867
22.376495361328125
8.1549654006958


In [18]:
m = statistics_tensor[14, :49]
m2 = statistics_tensor[5, :49]
l = statistics_tensor[210, :49]

distance12 = torch.norm(m-m2).item()
distance13 = torch.norm(m-l).item()
distance23 = torch.norm(m2-l).item()
print(distance12)
print(distance13)
print(distance23)

9.455707550048828
8.435786247253418
3.005241870880127


## Implementing new model now

In [11]:
class EmbeddingModel(nn.Module):
    def __init__(self):
        super(EmbeddingModel, self).__init__()
        self.layer1 = nn.Linear(150, 100)
        self.layer2 = nn.Linear(100, 75)
        self.layer3 = nn.Linear(75, 50)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

class FineTunedModel(nn.Module):
    def __init__(self, num_classes):
        super(FineTunedModel, self).__init__()
        self.base_model = EmbeddingModel()  # The pre-trained feature extractor
        self.classifier = nn.Linear(50, num_classes)  # New classification head

    def forward(self, x):
        x = self.base_model(x)
        x = self.classifier(x)
        return x


In [12]:
# Initialize the fine-tuned model with 47 classes (replace with the actual number of classes in your case)
fine_tuned_model = FineTunedModel(num_classes=47)

# Load the pre-trained weights into the base_model part of fine_tuned_model
fine_tuned_model.base_model.load_state_dict(torch.load('contrastive_model_HB.pth'))

# Move model to the device (CPU or CUDA)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)


FineTunedModel(
  (base_model): EmbeddingModel(
    (layer1): Linear(in_features=150, out_features=100, bias=True)
    (layer2): Linear(in_features=100, out_features=75, bias=True)
    (layer3): Linear(in_features=75, out_features=50, bias=True)
  )
  (classifier): Linear(in_features=50, out_features=47, bias=True)
)

In [28]:
from torch.nn import CrossEntropyLoss
import torch.optim as optim
# Your classification DataLoader
# classification_dataloader = DataLoader(your_classification_dataset, batch_size=32, shuffle=True)

# Define optimizers and loss function
optimizer_base = optim.Adam(fine_tuned_model.base_model.parameters(), lr=1e-4)
optimizer_classifier = optim.Adam(fine_tuned_model.classifier.parameters(), lr=1e-3)
classification_loss = CrossEntropyLoss()

# Training loop
num_epochs = 100 # Number of epochs for fine-tuning

# Initialize variables for tracking accuracy
total_samples = 0
correct_samples = 0

for epoch in range(num_epochs):
    for x_batch, y_batch in class_statistics_dataloader:  
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_batch = y_batch.squeeze()
        #print(y_batch.shape)
        # Forward pass
        outputs = fine_tuned_model(x_batch)
        #print(outputs.shape)
        # Compute loss
        loss = classification_loss(outputs, y_batch)
        
        # Zero gradients
        optimizer_base.zero_grad()
        optimizer_classifier.zero_grad()
        
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer_base.step()
        optimizer_classifier.step()
        
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_samples += y_batch.size(0)
        correct_samples += (predicted == y_batch).sum().item()

    # Calculate accuracy for this epoch
    epoch_accuracy = 100 * correct_samples / total_samples

    # Logging
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {epoch_accuracy:.2f}%')

    # Reset accuracy tracking variables for the next epoch
    total_samples = 0
    correct_samples = 0
    

Epoch [1/100], Loss: 2.5222, Accuracy: 34.17%
Epoch [2/100], Loss: 2.3246, Accuracy: 35.02%
Epoch [3/100], Loss: 2.6315, Accuracy: 35.60%
Epoch [4/100], Loss: 2.3418, Accuracy: 34.97%
Epoch [5/100], Loss: 2.3688, Accuracy: 35.18%
Epoch [6/100], Loss: 1.9851, Accuracy: 35.39%
Epoch [7/100], Loss: 2.1345, Accuracy: 36.14%
Epoch [8/100], Loss: 2.4829, Accuracy: 35.76%
Epoch [9/100], Loss: 2.2476, Accuracy: 35.76%
Epoch [10/100], Loss: 2.1818, Accuracy: 35.66%
Epoch [11/100], Loss: 2.1998, Accuracy: 35.92%
Epoch [12/100], Loss: 2.7314, Accuracy: 35.87%
Epoch [13/100], Loss: 2.3283, Accuracy: 35.98%
Epoch [14/100], Loss: 2.6070, Accuracy: 36.35%
Epoch [15/100], Loss: 2.2003, Accuracy: 36.56%
Epoch [16/100], Loss: 2.4004, Accuracy: 36.30%
Epoch [17/100], Loss: 2.4058, Accuracy: 36.14%
Epoch [18/100], Loss: 2.2293, Accuracy: 36.72%
Epoch [19/100], Loss: 2.4514, Accuracy: 36.72%
Epoch [20/100], Loss: 2.1385, Accuracy: 36.72%
Epoch [21/100], Loss: 2.7145, Accuracy: 36.56%
Epoch [22/100], Loss: 

Create dataset for testing

In [14]:
loading_transforms = transforms.Compose([
    transforms.CenterCrop(size=300),
    transforms.Grayscale(),
    transforms.ToTensor(),  # This will also convert the image from [0, 255] to [0.0, 1.0]
    transforms.ConvertImageDtype(torch.float32)
])

#use training set for now
test_dtd_dataset = torchvision.datasets.DTD(root='/home/gridsan/ckoevesdi/data/dtd_torch/', split='test', 
                                            partition=10, 
                                       transform=loading_transforms, target_transform=None,
                                       download=False) #ah das datenset muss so aussehen wie es auf der website auch ist, deswegen kann man auch download false machen
# Define the batch size (Change this based on your requirements)
batch_size = 1

# Create a DataLoader
test_classification_dataloader = DataLoader(
    test_dtd_dataset,
    batch_size=batch_size,
    shuffle=False
)
tensor2pil_transform = transforms.ToPILImage()
print(len(test_dtd_dataset))

517


In [15]:
print(len(test_dtd_dataset))

517


In [16]:
test_all_statistics = []

for j, (texture_batch, labels) in enumerate(test_classification_dataloader):
    # Assuming texture_batch has a shape of [20, 5, 1, 128, 128]
    #print(texture_batch.shape)
    output = texture_batch.to(device)
    output = torch.stack([output, output]).squeeze(1)
    #print(output.shape)
    #print(labels)
    statistics = brucy(output)  # This should output a tensor of shape [2, 150]
    statistics = statistics[0,:]
    #print(statistics[0,:])
    # You can now store these statistics, along with the label and original image index
    test_all_statistics.append({
        'statistics': statistics.cpu().numpy(),
        #'label': labels[i].item(),
        'original_image_index': labels.cpu().numpy()
    })

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# Convert the list of dictionaries to a NumPy array or PyTorch tensor
test_statistics_array = np.array([item['statistics'] for item in test_all_statistics])
test_labels_array = np.array([item['original_image_index'] for item in test_all_statistics])
print(test_statistics_array.shape)
# Standardize
scaler = StandardScaler()
test_scaled_array = scaler.fit_transform(test_statistics_array)

# Convert NumPy arrays to PyTorch tensors
test_statistics_tensor = torch.tensor(test_scaled_array, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels_array, dtype=torch.long)


(517, 150)


In [18]:
from torch.utils.data import Dataset, DataLoader

class StatisticsDataset(Dataset):
    def __init__(self, statistics_tensor, labels_tensor):
        self.statistics = statistics_tensor
        self.labels = labels_tensor
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.statistics[idx], self.labels[idx]

# Create the dataset object
test_statistics_dataset = StatisticsDataset(test_statistics_tensor, test_labels_tensor)

batches = 40
test_class_statistics_dataloader = DataLoader(test_statistics_dataset, batch_size=batches, shuffle=True)

# Perform testing

In [29]:
# Set the model to evaluation mode
fine_tuned_model.eval()

# Initialize variables for tracking accuracy
total_samples = 0
correct_samples = 0

In [30]:
fine_tuned_model.eval()

# Initialize variables for tracking accuracy
total_samples = 0
correct_samples = 0
all_preds = []
all_labels = []
# Disable gradient calculation
with torch.no_grad():
    for x_batch, y_batch in test_class_statistics_dataloader:  
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_batch = y_batch.squeeze()
        
        # Forward pass
        outputs = fine_tuned_model(x_batch)
        
        # Compute accuracy
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())
        total_samples += y_batch.size(0)
        correct_samples += (predicted == y_batch).sum().item()

# Calculate accuracy for the test set
test_accuracy = 100 * correct_samples / total_samples

print(f'Test Accuracy: {test_accuracy:.2f}%')

# Set the model back to training mode (optional if you continue training afterwards)
fine_tuned_model.train()


Test Accuracy: 13.54%


FineTunedModel(
  (base_model): EmbeddingModel(
    (layer1): Linear(in_features=150, out_features=100, bias=True)
    (layer2): Linear(in_features=100, out_features=75, bias=True)
    (layer3): Linear(in_features=75, out_features=50, bias=True)
  )
  (classifier): Linear(in_features=50, out_features=47, bias=True)
)

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
# Convert to NumPy arrays for easier slicing and indexing
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Generate the classification report
print(classification_report(all_labels, all_preds))

# Generate the confusion matrix
print(confusion_matrix(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00        11
           5       0.38      0.27      0.32        11
           6       0.14      0.09      0.11        11
           7       0.00      0.00      0.00        11
           8       0.12      0.09      0.11        11
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00        11
          11       0.20      0.09      0.13        11
          12       0.33      0.18      0.24        11
          13       0.08      0.45      0.14        11
          14       0.00      0.00      0.00        11
          15       0.17      0.36      0.24        11
          16       0.00      0.00      0.00        11
          17       0.00    

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh